In [1]:
import pandas as pd
import numpy as np
from transformers import pipeline
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification

In [2]:
all_sites = pd.read_csv('../data/all_sites.csv')

In [3]:
all_sites4 = all_sites[9300:12400]

In [4]:
# Select model
nlp_model = 'nlptown/bert-base-multilingual-uncased-sentiment'
tokenizer = AutoTokenizer.from_pretrained(nlp_model)
model = AutoModelForSequenceClassification.from_pretrained(nlp_model)

# Classifier
classifier = pipeline(
        'sentiment-analysis', 
        model=model, 
        tokenizer=tokenizer)

In [8]:
def sentiment_analysis_bert_base_multilingual_uncased(review):
    global_stars = []
    n = 1500
    
    review_splitted = [(review[i:i+n]) for i in range(0, len(review), n)]    
    global_stars = (classifier(review_splitted))
    
    return global_stars


In [9]:
def apply_sentiment_model(df):
    try:
        df['stars'] = df['text'].apply(lambda x: sentiment_analysis_bert_base_multilingual_uncased(x))
        
    except RuntimeError:
        print(df['game'])
        df['stars'] = df['text'].apply(lambda x: np.nan)
        
    finally:
        df.to_csv('../data/labeled_texts4.csv', index=False)
    
    return df

In [10]:
apply_sentiment_model(all_sites4)

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,site,url_link,author,game,company,genre,platform,text,score,stars
9300,Vandal,https://vandal.elespanol.com/analisis/ps4/the-...,Carlos Leiva,The Elder Scrolls Online: Tamriel Unlimited - ...,Bethesda,MMORPG,PS4 Xbox One,Hacerse un hueco a día de hoy en el mercado de...,6.0,"[{'label': '3 stars', 'score': 0.3550928235054..."
9301,3D Juegos,https://www.3djuegos.com/juegos/analisis/4501/...,Jesús Bella,Profesor Layton y el Futuro Perdido - Análisis,None,None,DS,El profesor Layton y su joven aprendiz Luke es...,8.5,"[{'label': '4 stars', 'score': 0.5195720791816..."
9302,Vandal,https://vandal.elespanol.com/analisis/ps4/man-...,Jorge Cano,Man of Medan - Análisis,Bandai Namco,Aventura Terror,PS4 Xbox One PC,En 2015 el estudio británico Supermassive Game...,7.8,"[{'label': '4 stars', 'score': 0.5135356783866..."
9303,Vandal,https://vandal.elespanol.com/analisis/psvita/d...,Ramón Varela,Disgaea 4: A Promise Revisited - Análisis,NIS America,Tactical RPG,PSVITA,"Los Disgaea salen primero en doméstica, pero c...",8.7,"[{'label': '4 stars', 'score': 0.4473478496074..."
9304,3D Juegos,https://www.3djuegos.com/juegos/analisis/9484/...,Sergio Martín,Tokyo Jungle - Análisis,None,None,PS3 Vita Android,"Después de debutar con éxito en Japón, Sony di...",7.5,"[{'label': '4 stars', 'score': 0.4288189411163..."
...,...,...,...,...,...,...,...,...,...,...
12395,3D Juegos,https://www.3djuegos.com/juegos/analisis/36567...,Alejandro Pascual,XCOM: Chimera Squad - Análisis,None,None,PC,"Como salido de una emboscada, lo de este video...",8.0,"[{'label': '4 stars', 'score': 0.5086400508880..."
12396,3D Juegos,https://www.3djuegos.com/juegos/analisis/31626...,Jesús Bella,Mario & Luigi: Viaje al centro de Bowser + Las...,None,None,· 3DS,"Nintendo 3DS tiene aún algo que decir, y lo ha...",9.0,"[{'label': '4 stars', 'score': 0.5369870662689..."
12397,Gamereactor,https://www.gamereactor.es/onward-para-oculus-...,Kalle Max Hofmann,Onward para Oculus Quest,Coatsink,Acción,Oculus Quest,"En 2016, salió Onward en PC fruto del trabajo ...",7.0,"[{'label': '4 stars', 'score': 0.3927950859069..."
12398,Vandal,https://vandal.elespanol.com/analisis/iphone/b...,Alberto González,Borderlands Legends - Análisis,2K Games,Action-RPG Mundo abierto Shooter en tercera pe...,iPhone,Borderlands es una de las sagas que mejor y má...,8.8,"[{'label': '4 stars', 'score': 0.5056893229484..."


In [11]:
def stars_mean_to_score(review):
    
    points = []

    for classification in review:

        #Mean of fragment
        grade = int(classification['label'].split(' ')[0])
        points.append(grade)  
    score = round(np.mean(points), 2)

    return(score)   

In [12]:
all_sites4['stars_mean'] = all_sites4['stars'].apply(lambda x: stars_mean_to_score(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [13]:
def score_labels(score):
    scorelabel = f"'{round(score / 2)} starts'"
    return scorelabel

In [14]:
all_sites4['score_label'] = all_sites4['score'].apply(lambda x: score_labels(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [15]:
def score_half(score):
    return score / 2

In [16]:
all_sites4['score_adj'] = all_sites4['score'].apply(lambda x: score_half(x))

/home/sergio/miniconda3/envs/lab_env/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [17]:
all_sites4.to_csv('../data/scored_texts4.csv', index=False)

In [18]:
all_sites4

,site,url_link,author,game,company,genre,platform,text,score,stars,stars_mean,score_label,score_adj
9300,Vandal,https://vandal.elespanol.com/analisis/ps4/the-...,Carlos Leiva,The Elder Scrolls Online: Tamriel Unlimited - ...,Bethesda,MMORPG,PS4 Xbox One,Hacerse un hueco a día de hoy en el mercado de...,6.0,"[{'label': '3 stars', 'score': 0.3550928235054...",2.67,'3 starts',3.00
9301,3D Juegos,https://www.3djuegos.com/juegos/analisis/4501/...,Jesús Bella,Profesor Layton y el Futuro Perdido - Análisis,None,None,DS,El profesor Layton y su joven aprendiz Luke es...,8.5,"[{'label': '4 stars', 'score': 0.5195720791816...",4.00,'4 starts',4.25
9302,Vandal,https://vandal.elespanol.com/analisis/ps4/man-...,Jorge Cano,Man of Medan - Análisis,Bandai Namco,Aventura Terror,PS4 Xbox One PC,En 2015 el estudio británico Supermassive Game...,7.8,"[{'label': '4 stars', 'score': 0.5135356783866...",3.56,'4 starts',3.90
9303,Vandal,https://vandal.elespanol.com/analisis/psvita/d...,Ramón Varela,Disgaea 4: A Promise Revisited - Análisis,NIS America,Tactical RPG,PSVITA,"Los Disgaea salen primero en doméstica, pero c...",8.7,"[{'label': '4 stars', 'score': 0.4473478496074...",3.67,'4 starts',4.35
9304,3D Juegos,https://www.3djuegos.com/juegos/analisis/9484/...,Sergio Martín,Tokyo Jungle - Análisis,None,None,PS3 Vita Android,"Después de debutar con éxito en Japón, Sony di...",7.5,"[{'label': '4 stars', 'score': 0.4288189411163...",3.20,'4 starts',3.75
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12395,3D Juegos,https://www.3djuegos.com/juegos/analisis/36567...,Alejandro Pascual,XCOM: Chimera Squad - Análisis,None,None,PC,"Como salido de una emboscada, lo de este video...",8.0,"[{'label': '4 stars', 'score': 0.5086400508880...",3.29,'4 starts',4.00
12396,3D Juegos,https://www.3djuegos.com/juegos/analisis/31626...,Jesús Bella,Mario & Luigi: Viaje al centro de Bowser + Las...,None,None,· 3DS,"Nintendo 3DS tiene aún algo que decir, y lo ha...",9.0,"[{'label': '4 stars', 'score': 0.5369870662689...",4.00,'4 starts',4.50
12397,Gamereactor,https://www.gamereactor.es/onward-para-oculus-...,Kalle Max Hofmann,Onward para Oculus Quest,Coatsink,Acción,Oculus Quest,"En 2016, salió Onward en PC fruto del trabajo ...",7.0,"[{'label': '4 stars', 'score': 0.3927950859069...",3.60,'4 starts',3.50
12398,Vandal,https://vandal.elespanol.com/analisis/iphone/b...,Alberto González,Borderlands Legends - Análisis,2K Games,Action-RPG Mundo abierto Shooter en tercera pe...,iPhone,Borderlands es una de las sagas que mejor y má...,8.8,"[{'label': '4 stars', 'score': 0.5056893229484...",3.67,'4 starts',4.40
